In [1]:
# import modules/libraries
import warnings 
warnings.simplefilter(action='ignore')
from entsoe import EntsoePandasClient
import pandas as pd
import json
import requests
from pandas import json_normalize
from bs4 import BeautifulSoup

import osmnx as ox
import pandas as pd
import numpy as np
import geopandas as gpd
import time
from datetime import datetime
from scipy import stats
import itertools
import os
import pickle
import geojson
from sqlalchemy import create_engine
import re
import sqlite3
from pathlib import Path
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
import chardet
from scipy import spatial
from scipy.spatial import KDTree
from shapely import wkt

cwd = Path().resolve()

# visualisation
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
import matplotlib as mpl 
%matplotlib inline 
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

node
  [man_made=pipeline]
  ({{bbox}});
out;

In [2]:
client = EntsoePandasClient(api_key="2d80d663-6eb6-447f-b31b-9d5c6b41365a")


In [3]:
s = requests.Session()
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

def lng_tankers():
    df2 = pd.DataFrame()
    for i in range(1,40):
        try:
            url = f'https://www.vesselfinder.com/de/vessels?page={i}&type=604'
            r = requests.get(url, headers=header)
            r.raise_for_status()
            df = pd.read_html(r.text, extract_links='all')[0]
            df.rename(columns={df.columns[0]: 'imo'}, inplace=True)
            df['imo'] = df.apply(lambda x: x['imo'][1], axis=1)
            df['imo']= df['imo'].str.split("/").str[-1]
            
            
            df.rename(columns={df.columns[1]: 'ship'},inplace=True)
            df['ship'] = df.apply(lambda x: x['ship'][0], axis=1)
    
            df.rename(columns={df.columns[2]: 'build'},inplace=True)
            df['build'] = df.apply(lambda x: x['build'][0], axis=1)
            
            df.rename(columns={df.columns[3]: 'GT'},inplace=True)
            df['GT'] = df.apply(lambda x: x['GT'][0], axis=1)

            df.rename(columns={df.columns[4]: 'DWT'},inplace=True)
            df['DWT'] = df.apply(lambda x: x['DWT'][0], axis=1)
            
            df.rename(columns={df.columns[5]: 'size'},inplace=True)
            df['size'] = df.apply(lambda x: x['size'][0], axis=1)
            
            df[['ship', 'type']] = df["ship"].apply(lambda x: pd.Series(str(x).split("  LNG Tanker")))
            df['type'] = "LNG Tanker"
            
            df2 = df2.append(df, ignore_index=True)
            
        except Exception as e:
            print(e)
            break
    
    df2.to_csv('./data/lng_tankers.csv', index=False)
    return df2


def get_location(imo):
    try:
        hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
           'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
           'Accept-Encoding': 'none',
           'Accept-Language': 'en-US,en;q=0.8',
           'Connection': 'keep-alive'}

        url = f"https://www.vesselfinder.com/en/vessels/VOS-TRAVELLER-IMO-{imo}"
        r = s.get(url, headers=header)
        r.raise_for_status()
        soup = BeautifulSoup(r.text)
        data = json.loads(soup.find("div", {"id": "djson"})['data-json'])
        print(data['ship_lat'], data['ship_lon'])
        time.sleep(0.1)
        return pd.Series([data['ship_lat'], data['ship_lon']])
    
    except Exception as e:
        print(e)
        input("change vpn")
        return pd.Series([np.nan, np.nan])

def get_vessels():
    # lng = lng_tankers()
    lng = pd.read_csv('./data/lng_tankers.csv', index_col=False)
    lng[['lat', 'lon']] = lng.apply(lambda x: get_location(x['imo']), axis=1)
    lng.to_csv('./data/lng_tankers_lat.csv', index=False)
    return df
# lng_vessesl = get_vessels()


https://github.com/EnergieID/entsoe-py/blob/master/entsoe/mappings.py

country codes and examples:
https://thesmartinsights.com/how-to-query-data-from-the-entso-e-transparency-platform-using-python/


In [4]:
start = pd.Timestamp('20221209', tz='Europe/Brussels')
end = pd.Timestamp('20221210', tz='Europe/Brussels')
country_code = 'AT'  # Belgium
country_code_from = 'FR'  # France
country_code_to = 'DE_LU' # Germany-Luxembourg
type_marketagreement_type = 'A01'
contract_marketagreement_type = 'A01'

data = {}


eu_list = ['AT', 'BE', 'BG','HR', 'CZ', 'DE_LU', 'DK', 'EE', 'FI', 'FR', 'GR', 'HU', 'IE', 'IT', 'LV',
          'NL']
for country in eu_list:
    try:
        s = client.query_day_ahead_prices(country, start=start,end=end)
        df = s.to_frame()
        df.rename(columns={df.columns[0]: 'price_mwh'},inplace=True)
        mean_price = df.price_mwh.mean()
        print(mean_price, country)
        #df.head(24)
        data[country] = mean_price
    except:
        print("problem in ", country)
    

problem in  AT
problem in  BE
problem in  BG
problem in  HR
problem in  CZ
problem in  DE_LU
problem in  DK
problem in  EE
problem in  FI
problem in  FR
problem in  GR
problem in  HU
problem in  IE
problem in  IT
problem in  LV
problem in  NL


Connection Error, retrying in 0 seconds
Connection Error, retrying in 0 seconds
Connection Error, retrying in 0 seconds
Connection Error, retrying in 0 seconds
Connection Error, retrying in 0 seconds
Connection Error, retrying in 0 seconds
Connection Error, retrying in 0 seconds
Connection Error, retrying in 0 seconds
Connection Error, retrying in 0 seconds
Connection Error, retrying in 0 seconds
Connection Error, retrying in 0 seconds
Connection Error, retrying in 0 seconds
Connection Error, retrying in 0 seconds
Connection Error, retrying in 0 seconds
Connection Error, retrying in 0 seconds
Connection Error, retrying in 0 seconds


In [5]:
data

{}

In [6]:
s = client.query_day_ahead_prices('IT_NORD', start=start,end=end)
df = s.to_frame()
df.rename(columns={df.columns[0]: 'price_mwh'},inplace=True)
mean_price = df.price_mwh.mean()
print(mean_price)
df.head(24)

Connection Error, retrying in 0 seconds


ConnectionError: HTTPSConnectionPool(host='web-api.tp.entsoe.eu', port=443): Max retries exceeded with url: /api?documentType=A44&in_Domain=10Y1001A1001A73I&out_Domain=10Y1001A1001A73I&securityToken=2d80d663-6eb6-447f-b31b-9d5c6b41365a&periodStart=202212072300&periodEnd=202212102300 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002E2F77BE910>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
start = pd.Timestamp('20221204', tz='Europe/Brussels')
end = pd.Timestamp('20221205', tz='Europe/Brussels')
t =client.query_generation_per_plant(country_code, start=start,end=end, psr_type=None)
t.head()

### Standorte

https://wiki.openmod-initiative.org/wiki/SciGRID_gas

maybe
http://users.dimi.uniud.it/~massimo.franceschet/ns/syllabus/make/gas/gas.html

https://github.com/mihai-craita/countries_center_box

In [ ]:
def get_centroid():
    """ Get centroid coordinates for all countries
        source: https://github.com/mihai-craita/countries_center_box
    """
    url = 'https://raw.githubusercontent.com/mihai-craita/countries_center_box/master/countries.json'
    r = requests.get(url)
    data = json.loads(r.text)
    df = pd.DataFrame.from_dict(data)
    return df

In [ ]:
def get_lat_long(point):
    """ get latitude and longitude coordinate from POINT geometry """
    try:
        return pd.Series([point.x, point.y])
    except:
        print("error")
        pass

def get_gie_data_V0():
    """ Read Excel data from gas infrastructure operators of Europe (GIE).
        source: https://www.gie.eu/transparency/databases/
                https://www.gas.scigrid.de/posts/2010-Okt-03_gie_raw.html
                
        just temporary -> later on use the exel file from GIE website
            #f = open('./data/GIE_Raw/data/GIE_LNGs.geojson')
    data = json.load(f)
    f.close()
    data = json.loads(data)
    df = pd.DataFrame.from_dict(data['features'][0])
    df = pd.read_json('./data/GIE_Raw/data/GIE_LNGs.geojson', orient='columns')
    """
    df = gpd.read_file('./data/GIE_Raw/data/GIE_LNGs.geojson')
    df[['lon', 'lat']] = df.apply(lambda x: get_lat_long(x['geometry']), axis=1)
    df.drop(columns=['tags', 'uncertainty', 'method', 'geometry', 'param'], inplace=True)
    return df

# GIE_LNGs = get_gie_data_V0()
# GIE_LNGs.tail(200)

In [ ]:
def add_trace(fig, df, col="#76b5c5", size=12):
    template=f"<br>Name: {df['Name']}</b><extra></extra>"
    fig.add_scattermapbox(
        lat=[df['lat']], lon=[df['lon']], mode='markers',
        hovertemplate= template,
        marker_size=size, marker_color=col,
        line=dict(width=12),opacity= 0.8, hoverinfo='none',
        showlegend=False,)


def draw_lng_terminals_V0(df):
    fig = go.Figure()
    
    for index, row in df.iterrows():
        add_trace(fig, row, col="#76b5c5", size=13)
    
    fig.update_layout(mapbox_style="light", mapbox_accesstoken="pk.eyJ1IjoiYXJlYnVyZyIsImEiOiJjbDZkZzdhOXAwNTk5M2ZteDdkY3lvd2FsIn0.c4hsFv69HozIVHq7ItrTzg",
                      mapbox_zoom=3,
                     mapbox_center={"lat": 47.809490, "lon": 13.055010})
    fig.update_layout(font=dict(family="Helvetica"), legend={"title": "Select category"})
    fig.update_layout(margin={"r": 0, "t": 40, "l": 0, "b": 0}, paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)')
    fig.update_layout(title_text = 'All global LNG Ports')
    fig.show()

In [ ]:
def get_gie_data_V1():
    """ Read Excel data from gas infrastructure operators of Europe (GIE).
        source: http://globalenergyobservatory.org/list.php?db=Transmission&type=LNG_Ports

    """
    df = pd.read_html('http://globalenergyobservatory.org/list.php?db=Transmission&type=LNG_Ports',
                      header=0, displayed_only=False, attrs = {'id': 'list'}, extract_links='all')[0]
    
    df.rename(columns={df.columns[0]:'Name', df.columns[1]:'Capacity', df.columns[2]:'Country',
                       df.columns[3]:'State'}, inplace=True)
    return df


def get_lat_long_V1(name):
    """ make request to separate link and extract lat and longitude. """
    url = f"http://globalenergyobservatory.org/{name}"
    df_tmp = pd.read_html(url, header=0, displayed_only=False)[0]
    for txt in df_tmp.columns:
        if 'Lat' in txt:
            res = [float(i) for i in re.findall(r"[-+]?(?:\d*\.*\d+)", txt) if not i.isdecimal()]
            try:
                return pd.Series([res[0], res[1]])
            except:
                pass

def radius_around_port(lat, lon, radius_in_meter):
    radius = gpd.GeoDataFrame({'ship': f'test', 'geometry': [Point(lon, lat)]}, crs='EPSG:4326')
    radius = radius.to_crs(epsg=7855).buffer(radius_in_meter).to_crs(epsg=4326)
    boundary_geojson = gpd.GeoSeries(radius)
    region = boundary_geojson.geometry.unary_union
    return region


def lng_ports():
    """ main function to scrape all lng ports """
    df = get_gie_data_V1()
    df[['lat', 'lon']] = df.apply(lambda x: get_lat_long_V1(x['Name'][1]), axis=1)
    df['Name'] = df.apply(lambda x: x['Name'][0], axis=1)
    df['Capacity'] = df.apply(lambda x: x['Capacity'][0], axis=1)
    df['Country'] = df.apply(lambda x: x['Country'][0], axis=1)
    df['State'] = df.apply(lambda x: x['State'][0], axis=1)
    df.to_csv('./data/GIE_LNGs_V1.csv', index=False)
    return df


def get_line(poly):
    try:
        #ls = shapely.geometry.LineString(df) # shapely.geometry.LineString(shapely.ops.unary_union(df).exterior.coords)
        b = poly.boundary.coords
        ls = shapely.geometry.LineString(b)
            
        #linestrings = [LineString(b[k:k+2]) for k in range(len(b) - 1)]
        #print([list(ls.coords) for ls in linestrings])
        lats, lons = ls.coords.xy
        return pd.Series([lats, lons])
    except Exception as e:
        print(e)
        return pd.Series([np.nan, np.nan])
    
# draw_lng_terminals_V0(GIE_LNGs_V1)
from shapely.geometry import Polygon, LineString

p = pd.read_csv('./data/GIE_LNGs_V1.csv', index_col=False) # lng_ports()
#p = p.iloc[10:50,:]
p['bubble'] = ports.apply(lambda x: radius_around_port(x['lat'], x['lon'], radius_in_meter=60000), axis=1)
#p=p.iloc[10:30,:]
p[['lats','lons']] = ports.apply(lambda x: get_line(x['bubble']), axis=1)
ports = p.copy()
ports.head(2)

In [ ]:
lng = pd.read_csv('./data/lng_tankers_lat.csv', index_col=False)
lng.sort_values(by='DWT', ascending=False, inplace=True)

### References Konkurrenz
https://lng.team/#prices
https://medium.com/analytics-vidhya/how-to-create-a-simple-web-app-for-a-vessels-eta-api-using-python-db485182a8c2


In [ ]:
ports = p.copy()
ports.head()
def check_if_coord_in_poly(region, lon, lat):
    """
    Check if a coordinate (lat,long) is within a given polygon
    Used to check for an address in vienna geometry
    source: https://stackoverflow.com/questions/48097742/geopandas-point-in-polygon
    Return: True if Point within Polygon
    
    source: https://stackoverflow.com/questions/36399381/whats-the-fastest-way-of-checking-if-a-point-is-inside-a-polygon-in-python
    
    """
    """
    _pnts = [Point(lon, lat)]
    poly = gpd.GeoSeries({'within': region})
    pnts = gpd.GeoDataFrame(geometry=_pnts, index=['Point to check'], crs=get_local_crs(lon, lat))
    pnts = pnts.assign(**{key: pnts.within(geom) for key, geom in poly.items()})
    return pnts['within'].item()
    """
    point = Point(lon, lat)
    return region.contains(point)
    
for index, row in ports.iterrows():
    lng['close_to_harbor'] = lng.apply(lambda x: check_if_coord_in_poly(row['bubble'], x['lon'], x['lat']), axis=1)

#lng['vessel'] = lng.apply(lambda x: harbor(x['ship'], x['lat'], x['lon']), axis=1)
lng.head()

In [ ]:
def add_ship(fig, df, col="#76b5c5", size=12):
    try:
        template=f"<br>Name: {df['imo']}</b><extra></extra>"
        fig.add_scattermapbox(
            lat=[df['lat']], lon=[df['lon']], mode='markers',
            hovertemplate=template,
            marker_size=size, marker_color=col,
            line=dict(width=6),opacity= 0.4, hoverinfo='none',
            showlegend=False,)
    
    except Exception as e:
        print(e)

def add_radius(fig, df):
    try:
        radius_in_meter = 10000
        fig.add_trace(go.Scattermapbox(
                mode="lines",
                lon=list(df['lats']),
                lat=list(df['lons']),
                name=f"{radius_in_meter} m radius",
                hoverinfo='skip',
                showlegend=False,
                marker={'size': 15, 'color': 'red', 'opacity': 0.2}))
    except Exception as e:
        pass

def add_ports(fig, df, col="#bbb5c5", symbol="harbor", size=12):
    try:
        if symbol == "harbor":
            fig.add_scattermapbox(
            name='Home', lat=df['lat'].tolist(), lon=df['lon'].tolist(),
            mode='markers+text',
            marker=dict(size=size, color=col),
            text=df['Name'].tolist(),
            textposition="bottom center",
            hoverinfo='text',
            textfont=dict(color='black', size=10),
            showlegend=False,)
                
        else:
            #template=f"<br>Name: {df['imo']}</b><extra></extra>"
            fig.add_scattermapbox(
                name='Home',
                lat=df['lat'].tolist(), lon=df['lon'].tolist(),
                mode='markers',
                #hovertemplate=template,
                #marker_size=size, 
                #marker_color=col,
                marker=dict(size=size, symbol=symbol, color=col),
                #line=dict(width=6),
                #if symbol == "harbor":
                #    text=df['Name'].tolist(),
               #    textposition="bottom center",
                #    hoverinfo='text',
               #     textfont=dict(color='black', size=10),

                showlegend=False,)

    except Exception as e:
        print(e)

def vessel(df, ports):
    fig = go.Figure()
    
    
    add_ports(fig, ports, col="#76b5c5", symbol="harbor", size=16)
    add_ports(fig, df, col="#be4d25", symbol="marker", size=10)   
    
        
    for index, row in ports.iterrows():
        add_radius(fig, row)
    
    fig.update_layout(mapbox_style="light", mapbox_accesstoken="pk.eyJ1IjoiYXJlYnVyZyIsImEiOiJjbDZkZzdhOXAwNTk5M2ZteDdkY3lvd2FsIn0.c4hsFv69HozIVHq7ItrTzg",
                      mapbox_zoom=3,
                     mapbox_center={"lat": 47.809490, "lon": 13.055010})
    fig.update_layout(font=dict(family="Helvetica"), legend={"title": "Select category"})
    fig.update_layout(margin={"r": 0, "t": 40, "l": 0, "b": 0}, paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)')
    fig.update_layout(title_text = 'All global LNG Ports')
    fig.show()


vessel(lng, ports)

In [ ]:
from pathlib import Path
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon

gdf = gpd.GeoDataFrame({'address': 'Burggasse 2, Wien', 'geometry': [Point(16.363449, 48.210033)]}, crs='EPSG:4326')
gdf.head()

In [ ]:
radius = gdf.copy()
radius_in_meter = 1000
radius = radius.to_crs(epsg=7855).buffer(radius_in_meter).to_crs(epsg=4326)
radius.head()
import geopandas as gpd

In [ ]:
shapely_polygon= shapely.geometry.Polygon([(0, 0), (0, 1), (1, 0)])
t = gpd.GeoSeries(radius).to_json()
t

In [ ]:
import shapely.ops
import shapely.geometry
ls = shapely.geometry.LineString(shapely.ops.unary_union(radius).exterior.coords)
ls
lats, lons = ls.coords.xy
lats

In [ ]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

def get_region():
    """  """
    boundary_geojson = gpd.read_file(os.path.join(Path(cwd), 'data', 'geojson', 'vienna.geojson'))
    boundary_geojson.drop(columns=['cartodb_id', 'created_at', 'updated_at'], inplace=True)
    region = boundary_geojson.geometry.unary_union
    return region
    
def get_local_crs(y, x):
    """ get local crs """
    x = ox.utils_geo.bbox_from_point((y, x), dist=500, project_utm=True, return_crs=True)
    return x[-1]




#if not check_if_coord_in_poly(region, dfi['longitude'].item(), dfi['latitude'].item()):

In [ ]:
ports.head()

In [ ]:
geome = ports['bubble'].iloc[0]
ls = shapely.geometry.LineString(shapely.ops.unary_union(geome).exterior.coords)
lats, lons = ls.coords.xy
lats

In [ ]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

point = Point(0.5, 2.5)
polygon = Polygon([(0, 0), (0, 1), (1, 1), (1, 0)])
print(polygon.contains(point))

In [ ]:
polygon
